In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
from sklearn.model_selection import KFold
from torchvision.models import GoogLeNet, GoogLeNet_Weights
import numpy as np
from sklearn.metrics import precision_score, f1_score, confusion_matrix

In [3]:
# pre-processing the data
# First Transformer applied to Lenet-5

# transform = transforms.Compose([
#     transforms.Resize((32,32)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
# ])

# Second Transformer applied to VGG16

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])


dataset = datasets.ImageFolder(root='E:\\Temp\\test', transform=transform)

In [4]:
dataset.class_to_idx

{'Fake': 0, 'Real': 1}

In [5]:
kfolds = KFold(n_splits=3, shuffle=True, random_state=42)

In [6]:
import torch.nn as nn
import torch.optim as optim

class Le_Net5(nn.Module):
    def __init__(self, num_classes=2):
        super(Le_Net5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fcl1 = nn.Linear(16*5*5, 120)
        self.fcl2 = nn.Linear(120, 84)
        self.fcl3 = nn.Linear(84, num_classes)

    def forward(self, X):
        X = torch.relu(self.conv1(X))
        X = torch.max_pool2d(X, 2)
        X = torch.relu(self.conv2(X))
        X = torch.max_pool2d(X, 2)
        X = X.view(X.size(0), -1) # flatten the tensor
        X = torch.relu(self.fcl1(X))
        X = torch.relu(self.fcl2(X))
        X = self.fcl3(X)
        return X
    

model = Le_Net5(num_classes=2)

In [7]:
dataset_labels = np.array([data[1] for data in dataset])

for fold, (train_idx, test_idx) in enumerate(kfolds.split(dataset, dataset_labels)):
    train_val_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    train_size = int(0.8 * len(dataset))
    val_size = len(train_val_subset) - train_size
    train_split, val_split = random_split(train_val_subset, [train_size, val_size])

    train_loader = DataLoader(train_split, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_split, batch_size=64, shuffle=False)
    test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Fold {fold + 1}, Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

    model.eval()
    val_labels = []
    val_preds = []
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predicted.cpu().numpy())
    
    val_precision = precision_score(val_labels, val_preds, average='binary')
    val_f1_score = f1_score(val_labels, val_preds, average='binary')
    val_confusion_matrix = confusion_matrix(val_labels, val_preds)
    print(f"Fold {fold + 1}, Validation Precision: {val_precision:.2f}%")
    print(f"Fold {fold + 1}, Validation F1 Score: {val_f1_score:.2f}%")
    print(f"Fold {fold + 1}, Validatoin Confusion Matrix\n {val_confusion_matrix}")


    test_labels = []
    test_preds = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_labels.extend(labels.cpu().numpy())
            test_preds.extend(predicted.cpu().numpy())

    
    test_precision = precision_score(test_labels, test_preds, average='binary')
    test_f1_score = f1_score(test_labels, test_preds, average='binary')
    test_confusion_matrix = confusion_matrix(test_labels, test_preds)
    
    print(f"Fold {fold + 1}, Test Precision: {test_precision:.2f}%")
    print(f"Fold {fold + 1}, Test F1 Score {test_f1_score:.2f}%")
    print(f"Fold {fold + 1}, Test Confusion Matrix\n {test_confusion_matrix}")

Fold 1, Epoch 1, Loss: 0.6245187259556955
Fold 1, Epoch 2, Loss: 0.5604676243505979
Fold 1, Epoch 3, Loss: 0.5304904164452302
Fold 1, Epoch 4, Loss: 0.5034905452477304
Fold 1, Epoch 5, Loss: 0.4841533240519072
Fold 1, Epoch 6, Loss: 0.4531817810054411
Fold 1, Epoch 7, Loss: 0.43091117263885964
Fold 1, Epoch 8, Loss: 0.40682335067213626
Fold 1, Epoch 9, Loss: 0.38768051146415244
Fold 1, Epoch 10, Loss: 0.36950460376969557


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fold 1, Validation Precision: 0.00%
Fold 1, Validation F1 Score: 0.00%
Fold 1, Validatoin Confusion Matrix
 []
Fold 1, Test Precision: 0.73%
Fold 1, Test F1 Score 0.76%
Fold 1, Test Confusion Matrix
 [[1340  524]
 [ 372 1399]]
Fold 2, Epoch 1, Loss: 0.4281153584781446
Fold 2, Epoch 2, Loss: 0.3930619347252344
Fold 2, Epoch 3, Loss: 0.363577891598668
Fold 2, Epoch 4, Loss: 0.3324931746250705
Fold 2, Epoch 5, Loss: 0.2953942751413898
Fold 2, Epoch 6, Loss: 0.2673659320724638
Fold 2, Epoch 7, Loss: 0.2340891337708423
Fold 2, Epoch 8, Loss: 0.19581028819084167
Fold 2, Epoch 9, Loss: 0.17148278124238314
Fold 2, Epoch 10, Loss: 0.1281999667550911
Fold 2, Validation Precision: 0.00%
Fold 2, Validation F1 Score: 0.00%
Fold 2, Validatoin Confusion Matrix
 []


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fold 2, Test Precision: 0.80%
Fold 2, Test F1 Score 0.81%
Fold 2, Test Confusion Matrix
 [[1475  359]
 [ 335 1466]]
Fold 3, Epoch 1, Loss: 0.3243685670589146
Fold 3, Epoch 2, Loss: 0.2416359703791769
Fold 3, Epoch 3, Loss: 0.20483815591586263
Fold 3, Epoch 4, Loss: 0.16377228531136848
Fold 3, Epoch 5, Loss: 0.14189488519179194
Fold 3, Epoch 6, Loss: 0.105197360510366
Fold 3, Epoch 7, Loss: 0.08822011248322956
Fold 3, Epoch 8, Loss: 0.06972953805438521
Fold 3, Epoch 9, Loss: 0.059846046282664725
Fold 3, Epoch 10, Loss: 0.0647745308703171
Fold 3, Validation Precision: 0.00%
Fold 3, Validation F1 Score: 0.00%
Fold 3, Validatoin Confusion Matrix
 []


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fold 3, Test Precision: 0.87%
Fold 3, Test F1 Score 0.89%
Fold 3, Test Confusion Matrix
 [[1539  255]
 [ 148 1693]]


In [8]:
import random

def optimized(model, lrates, train_loader, test_loader, criterion, device):
    best_f1 = 0
    best_precision = 0
    best_conf_matrix = None
    best_lr = None

    for lr in random.sample(lrates, len(lrates)):
        print(f"{lr} currently testing")

        model = model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        model.train()
        for epoch in range(1):
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        f1 = f1_score(all_targets, all_preds, average='macro')
        precision = precision_score(all_targets, all_preds, average='macro')
        conf_matrix = confusion_matrix(all_targets, all_preds)

        print(f"Learning rate : {lr}, f1-score: {f1:.2f}, precision: {precision:.2f}")

        if f1 > best_f1:
            best_f1 = f1
            best_precision = precision
            best_conf_matrix = conf_matrix
            best_lr = lr

    return best_f1, best_precision, best_conf_matrix, best_lr

In [9]:
dataset

Dataset ImageFolder
    Number of datapoints: 10905
    Root location: E:\Temp\test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [10]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
best_f1, best_precision, best_conf_matrix, best_lr = optimized(
    model = model,
    lrates = [0.0001,0.001,0.01,0.1],
    train_loader = train_loader,
    test_loader = test_loader,
    criterion=nn.CrossEntropyLoss(),
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

0.01 currently testing
Learning rate : 0.01, f1-score: 0.71, precision: 0.71
0.001 currently testing
Learning rate : 0.001, f1-score: 0.75, precision: 0.75
0.0001 currently testing
Learning rate : 0.0001, f1-score: 0.75, precision: 0.76
0.1 currently testing
Learning rate : 0.1, f1-score: 0.34, precision: 0.26


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
print(f"Best Learning Rate: {best_lr}")
print(f"Best F1-Score: {best_f1}")
print(f"Best Precision: {best_precision}")
print(f"Confusion Matrix:\n{best_conf_matrix}")

Best Learning Rate: 0.001
Best F1-Score: 0.7285218094112712
Best Precision: 0.7471600845104602
Confusion Matrix:
[[671 426]
 [158 926]]


In [12]:
model 

Le_Net5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fcl1): Linear(in_features=400, out_features=120, bias=True)
  (fcl2): Linear(in_features=120, out_features=84, bias=True)
  (fcl3): Linear(in_features=84, out_features=2, bias=True)
)

# Pretrained models

In [7]:
from torchvision import models

model_vgg = models.vgg16(pretrained=True)
model_vgg

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
# Make a train loader and test loader for vgg16 model

train_size = int(len(dataset) * 0.8)
test_size = int(len(dataset) - train_size)

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset,batch_size=34, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [13]:
model_vgg.classifier[6] = nn.Linear(4096, 2) # 2 is the number of classes we have in our dataset

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_vgg.parameters(),lr=0.001)

In [ ]:
# Train

num_epochs = 3
for epoch in range(num_epochs):
    model_vgg.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_vgg(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


model_vgg.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model_vgg(images)
        _, predicted = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(predicted.cpu().numpy())

precision = precision_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
conf_matrix = confusion_matrix(test_labels, test_preds)
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"Confusion Matris: \n{conf_matrix}")